# nuScenes dataset - Corruption

In [1]:
# imports

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud

## nuscenes:
- 0    CAM_FRONT,
- 1    CAM_FRONT_RIGHT,
- 2    CAM_FRONT_LEFT,
- 3    CAM_BACK,
- 4    CAM_BACK_LEFT,
- 5    CAM_BACK_RIGHT

In [14]:
class NuScenesCorruptionDataset():
    def __init__(self) -> None:
        self.nusc_info = NuScenes(version='v1.0-mini', dataroot='/home/bk/study/datasets/nuscenes_mini/', verbose=False)
        self.camera_id = 0
        self.scenes = self.nusc_info.scene

        self.frame_counts = 0
        self.samples = list()
        self.scenes = self.nusc_info.scene

        self.frame_counts = 0
        self.seq_counts = len(self.scenes)

        print(f'Found {self.seq_counts} sequences')

        for scene in self.scenes:
            # Get the first sample token from the scene
            sample_token = scene['first_sample_token']

            while sample_token:
                self.samples.append(sample_token)
                sample_record = self.nusc_info.get('sample', sample_token)
                self.frame_counts += 1
                sample_token = sample_record['next'] if sample_record['next'] else None

        print(f'Found {self.frame_counts} frames')

    
    def __len__(self):
        return self.frame_counts
    
        

        


In [15]:
n = NuScenesCorruptionDataset()

Found 10 sequences
Found 404 frames


In [16]:
import os
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion
import json

# Define the path to the NuScenes dataset
data_root = '/home/bk/study/datasets/nuscenes_mini_corrupted/'
nusc = NuScenes(version='v1.0-mini', dataroot=data_root, verbose=True)

# Function to miscalibrate the lidar to camera transformation
def miscalibrate_lidar2cam(transform_matrix, translation_offset=(0.1, 0.1, 0.1), rotation_offset=(0.01, 0.01, 0.01)):
    """
    Miscalibrate the lidar to camera transform by applying translation and rotation offsets.
    
    :param transform_matrix: Original transformation matrix (4x4).
    :param translation_offset: Tuple of (x, y, z) translation offsets.
    :param rotation_offset: Tuple of (yaw, pitch, roll) rotation offsets in radians.
    :return: Miscalibrated transformation matrix (4x4).
    """
    # Extract original rotation and translation
    orig_translation = transform_matrix[:3, 3]
    orig_rotation = Quaternion(matrix=transform_matrix[:3, :3])

    # Apply translation offset
    new_translation = orig_translation + np.array(translation_offset)

    # Apply rotation offset
    yaw, pitch, roll = rotation_offset
    delta_rotation = Quaternion(axis=[0, 0, 1], angle=yaw) * Quaternion(axis=[0, 1, 0], angle=pitch) * Quaternion(axis=[1, 0, 0], angle=roll)
    new_rotation = orig_rotation * delta_rotation

    # Create new transformation matrix
    new_transform = transform_matrix.copy()
    new_transform[:3, 3] = new_translation
    new_transform[:3, :3] = new_rotation.rotation_matrix

    return new_transform

# Iterate through each sample in the dataset
for sample in nusc.sample:
    sample_data = nusc.get('sample_data', sample['data']['LIDAR_TOP'])

    # Get the original lidar2cam transformation
    cs_record = nusc.get('calibrated_sensor', sample_data['calibrated_sensor_token'])
    lidar2cam_transform = transform_matrix(cs_record['translation'], Quaternion(cs_record['rotation']), inverse=False)

    # Miscalibrate the lidar2cam transformation
    miscalibrated_transform = miscalibrate_lidar2cam(lidar2cam_transform)

    # Update the calibration record with the miscalibrated transformation
    cs_record['translation'] = miscalibrated_transform[:3, 3].tolist()
    cs_record['rotation'] = Quaternion(matrix=miscalibrated_transform[:3, :3]).elements.tolist()

    # Save the modified calibration record
    cs_record_path = os.path.join(data_root, 'v1.0-mini', 'calibrated_sensor', f"{cs_record['token']}.json")
    with open(cs_record_path, 'w') as f:
        json.dump(cs_record, f)

    print(f"Updated calibration for sample: {sample['token']}")

print("Finished miscalibrating and saving the data.")

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.677 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


FileNotFoundError: [Errno 2] No such file or directory: '/home/bk/study/datasets/nuscenes_mini_corrupted/v1.0-mini/calibrated_sensor/a183049901c24361a6b0b11b8013137c.json'

In [18]:
import os
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion
import json

# Define the path to the NuScenes dataset
data_root = '/home/bk/study/datasets/nuscenes_mini'
output_root = '/home/bk/study/datasets/nuscenes_mis_calibrated'
os.makedirs(output_root, exist_ok=True)
nusc = NuScenes(version='v1.0-mini', dataroot=data_root, verbose=True)

# Function to miscalibrate the lidar to camera transformation
def miscalibrate_lidar2cam(transform_matrix, translation_offset=(0.1, 0.1, 0.1), rotation_offset=(0.01, 0.01, 0.01)):
    """
    Miscalibrate the lidar to camera transform by applying translation and rotation offsets.
    
    :param transform_matrix: Original transformation matrix (4x4).
    :param translation_offset: Tuple of (x, y, z) translation offsets.
    :param rotation_offset: Tuple of (yaw, pitch, roll) rotation offsets in radians.
    :return: Miscalibrated transformation matrix (4x4).
    """
    # Extract original rotation and translation
    orig_translation = transform_matrix[:3, 3]
    orig_rotation = Quaternion(matrix=transform_matrix[:3, :3])

    # Apply translation offset
    new_translation = orig_translation + np.array(translation_offset)

    # Apply rotation offset
    yaw, pitch, roll = rotation_offset
    delta_rotation = Quaternion(axis=[0, 0, 1], angle=yaw) * Quaternion(axis=[0, 1, 0], angle=pitch) * Quaternion(axis=[1, 0, 0], angle=roll)
    new_rotation = orig_rotation * delta_rotation

    # Create new transformation matrix
    new_transform = transform_matrix.copy()
    new_transform[:3, 3] = new_translation
    new_transform[:3, :3] = new_rotation.rotation_matrix

    return new_transform

# Dictionary to store all modified calibration records
modified_calibrated_sensors = {}

# Iterate through each sample in the dataset
for sample in nusc.sample:
    sample_data = nusc.get('sample_data', sample['data']['LIDAR_TOP'])

    # Get the original lidar2cam transformation
    cs_record = nusc.get('calibrated_sensor', sample_data['calibrated_sensor_token'])
    lidar2cam_transform = transform_matrix(cs_record['translation'], Quaternion(cs_record['rotation']), inverse=False)

    # Miscalibrate the lidar2cam transformation
    miscalibrated_transform = miscalibrate_lidar2cam(lidar2cam_transform)

    # Update the calibration record with the miscalibrated transformation
    cs_record['translation'] = miscalibrated_transform[:3, 3].tolist()
    cs_record['rotation'] = Quaternion(matrix=miscalibrated_transform[:3, :3]).elements.tolist()

    # Add the modified calibration record to the dictionary
    modified_calibrated_sensors[cs_record['token']] = cs_record

    print(f"Updated calibration for sample: {sample['token']}")

# Save the modified calibration records to a new calibrated_sensor.json file
output_calibrated_sensor_path = os.path.join(output_root, 'calibrated_sensor.json')
with open(output_calibrated_sensor_path, 'w') as f:
    json.dump(modified_calibrated_sensors, f)

print(f"Finished miscalibrating and saving the data to {output_calibrated_sensor_path}.")


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.587 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
Updated calibration for sample: ca9a282c9e77460f8360f564131a8af5
Updated calibration for sample: 39586f9d59004284a7114a68825e8eec
Updated calibration for sample: 356d81f38dd9473ba590f39e266f54e5
Updated calibration for sample: e0845f5322254dafadbbed75aaa07969
Updated calibration for sample: c923fe08b2ff4e27975d2bf30934383b
Updated calibration for sample: f1e3d9d08f044c439ce86a2d6fcca57b
Updated calibration for sample: 4f545737bf3347fbbc9af60b0be9a963
Updated calibration for sample: 7626dde27d604ac28a0240bdd54eba7a
Updated calibration for sample: be99ffc878b24aca8956bbb4e0f97d0c
Updated calibration for sample: 9813c23a5f1448b09bb7910fea9baf20
Updated calibration for sam